In [1]:
import csv
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from math import sqrt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.preprocessing import normalize
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [2]:
df_nyclistings = pd.read_csv('/Users/laranahcivan/Desktop/ECON 1680/ECON 1680 Project 1 Data/NYC Listings.csv')
print(df_nyclistings)

                       id                                              name  \
0      816783428767938211                    Cool studio 3 min from subway!   
1                 8686040                       LARGE MANHATTAN ONE BEDROOM   
2                12426778                                         TH BRKLYN   
3                22671517                                     55 washington   
4                35776956   luxury apt in long lsland city \n1min to subway   
...                   ...                                               ...   
39314            38023200  Private bedroom in artsy, light-filled apartment   
39315            38704132   Trendy Location ❤️ Private Room with Queen bed!   
39316            32419542                   JFK Airport delight non smoking   
39317            32427478                                     Brooklyn home   
39318            36231329           Large Designer One Bedroom on West 72nd   

         host_id host_name neighbourhood_group     

In [3]:
df_sflistings = pd.read_csv('/Users/laranahcivan/Desktop/ECON 1680/ECON 1680 Project 1 Data/San Francisco Listings.csv')
print(df_sflistings)

                      id                                             name  \
0                1094764                 San Francisco Presidio Paradise!   
1               38047206                                    52 Vesta home   
2               43475468     Beautiful Mission District Home and Backyard   
3     648549709021440854    LuxoStays | ! Quiet Rm #Private Bathrm & VIEW   
4               47918229      Blueground | Marina District, w/d, nr parks   
...                  ...                                              ...   
8356            40191004    #lF43 Private room for 1-2 ppl hacknsleep,com   
8357            32766699       Luxury upper flat Marina, 1 block from Bay   
8358            25842694             A cozy private unit in San Francisco   
8359               47682                 One-bedroom apt.  Lafayette Park   
8360            29039496  Excellent 1 Bedroom Hotel Suite in Union Square   

        host_id   host_name  neighbourhood_group          neighbourhood  \


In [4]:
print(f"Number of missing prices in df_sflistings: {df_sflistings['price'].isna().sum()}")
print(f"Number of missing prices in df_nyclistings: {df_nyclistings['price'].isna().sum()}")

Number of missing prices in df_sflistings: 1587
Number of missing prices in df_nyclistings: 15042


In [5]:
print(f"Total number of listings in df_sflistings: {df_sflistings.shape[0]}")
print(f"Total number of listings in df_nyclistings: {df_nyclistings.shape[0]}")

Total number of listings in df_sflistings: 8361
Total number of listings in df_nyclistings: 39319


In [6]:
print(f"Ratio of missing prices to total number of listings in SF: {df_sflistings['price'].isna().sum()/df_sflistings.shape[0]}")
print(f"Ratio of missing prices to total number of listings in NYC: {df_nyclistings['price'].isna().sum()/df_nyclistings.shape[0]}")

Ratio of missing prices to total number of listings in SF: 0.1898098313598852
Ratio of missing prices to total number of listings in NYC: 0.38256313741448156


In [7]:
#for viewing the columns with NaN values in df_sflistings
print("Columns with NaN values in df_sflistings:")
print(df_sflistings.columns[df_sflistings.isna().any()].tolist())

#for viewing the columns with NaN values in df_nyclistings
print("\nColumns with NaN values in df_nyclistings:")
print(df_nyclistings.columns[df_nyclistings.isna().any()].tolist())

Columns with NaN values in df_sflistings:
['neighbourhood_group', 'price', 'last_review', 'reviews_per_month', 'license']

Columns with NaN values in df_nyclistings:
['name', 'host_name', 'price', 'last_review', 'reviews_per_month', 'license']


In [8]:
#dropping the listings that do not contain "price"
df_sflistings = df_sflistings.dropna(subset=['price'])
df_nyclistings = df_nyclistings.dropna(subset=['price'])

#dropping the following columns: longitude, latitude, id, host_id, name, host_name, license, last_review, reviews_per_month, neighbourhood_group
df_sflistings = df_sflistings.drop(['longitude', 'latitude', 'id', 'host_id', 'name', 'host_name', 'license', 'last_review', 'reviews_per_month', 'neighbourhood_group'], axis=1)
df_nyclistings = df_nyclistings.drop(['longitude', 'latitude', 'id', 'host_id', 'name', 'host_name', 'license', 'last_review', 'reviews_per_month', 'neighbourhood_group'], axis=1)

In [9]:
print(f"Total number of listings in df_sflistings: {df_sflistings.shape[0]}")
print(f"Total number of listings in df_nyclistings: {df_nyclistings.shape[0]}")

Total number of listings in df_sflistings: 6774
Total number of listings in df_nyclistings: 24277


In [10]:
#for my own viewing purposes
df_nyclistings
df_sflistings

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,Inner Richmond,Entire home/apt,150.0,30,1,2,365,0
3,Excelsior,Private room,69.0,30,6,13,216,3
4,Marina,Entire home/apt,246.0,31,0,183,341,0
5,Presidio Heights,Entire home/apt,328.0,5,12,1,277,12
6,Castro/Upper Market,Private room,135.0,30,7,2,172,2
...,...,...,...,...,...,...,...,...
8356,Mission,Private room,50.0,365,0,55,365,0
8357,Marina,Entire home/apt,283.0,30,3,2,340,0
8358,Visitacion Valley,Entire home/apt,68.0,30,25,3,50,2
8359,Pacific Heights,Entire home/apt,108.0,30,35,9,335,1


In [11]:
#for viewing the data types I have in both of my data frames
print("NYC Data-Frame Data Types:")
print(df_nyclistings.dtypes)
print() #inserting a line
print("SF Data-Frame Data Types:")
print(df_sflistings.dtypes)

NYC Data-Frame Data Types:
neighbourhood                      object
room_type                          object
price                             float64
minimum_nights                      int64
number_of_reviews                   int64
calculated_host_listings_count      int64
availability_365                    int64
number_of_reviews_ltm               int64
dtype: object

SF Data-Frame Data Types:
neighbourhood                      object
room_type                          object
price                             float64
minimum_nights                      int64
number_of_reviews                   int64
calculated_host_listings_count      int64
availability_365                    int64
number_of_reviews_ltm               int64
dtype: object


In [12]:
#converting integer variables to float variables in both dataframes
integer_to_float_columns = ['minimum_nights', 'number_of_reviews', 'calculated_host_listings_count', 'availability_365', 'number_of_reviews_ltm']
df_nyclistings[integer_to_float_columns] = df_nyclistings[integer_to_float_columns].astype(float)
df_sflistings[integer_to_float_columns] = df_sflistings[integer_to_float_columns].astype(float)

#performing one-hot encoding on object variables in both dataframes
df_nyclistings = pd.get_dummies(df_nyclistings, columns=['neighbourhood', 'room_type'], drop_first=True)
df_sflistings = pd.get_dummies(df_sflistings, columns=['neighbourhood', 'room_type'], drop_first=True)

#viewing the data types of both data-frames to ensure everything is correct
print("NYC Data-Frame Data Types:")
print(df_nyclistings.dtypes)
print() #inserting a line
print("SF Data-Frame Data Types:")
print(df_sflistings.dtypes)

NYC Data-Frame Data Types:
price                             float64
minimum_nights                    float64
number_of_reviews                 float64
calculated_host_listings_count    float64
availability_365                  float64
                                   ...   
neighbourhood_Woodrow                bool
neighbourhood_Woodside               bool
room_type_Hotel room                 bool
room_type_Private room               bool
room_type_Shared room                bool
Length: 232, dtype: object

SF Data-Frame Data Types:
price                                  float64
minimum_nights                         float64
number_of_reviews                      float64
calculated_host_listings_count         float64
availability_365                       float64
number_of_reviews_ltm                  float64
neighbourhood_Bernal Heights              bool
neighbourhood_Castro/Upper Market         bool
neighbourhood_Chinatown                   bool
neighbourhood_Crocker Amazon      

In [13]:
#converting my boolean variables into float variables

#for NYC listings first
boolean_columns_nyc = df_nyclistings.select_dtypes(include=bool).columns
df_nyclistings[boolean_columns_nyc] = df_nyclistings[boolean_columns_nyc].astype(float)

#now for SF listings
boolean_columns_sf = df_sflistings.select_dtypes(include=bool).columns
df_sflistings[boolean_columns_sf] = df_sflistings[boolean_columns_sf].astype(float)

In [14]:
#checking for missing values

#for NYC listings first
missing_values_nyc = df_nyclistings.isnull().sum()
print("Missing values in NYC Listings:")
print(missing_values_nyc)

#now for SF listings
missing_values_sf = df_sflistings.isnull().sum()
print("\nMissing values in SF Listings:")
print(missing_values_sf)

Missing values in NYC Listings:
price                             0
minimum_nights                    0
number_of_reviews                 0
calculated_host_listings_count    0
availability_365                  0
                                 ..
neighbourhood_Woodrow             0
neighbourhood_Woodside            0
room_type_Hotel room              0
room_type_Private room            0
room_type_Shared room             0
Length: 232, dtype: int64

Missing values in SF Listings:
price                                  0
minimum_nights                         0
number_of_reviews                      0
calculated_host_listings_count         0
availability_365                       0
number_of_reviews_ltm                  0
neighbourhood_Bernal Heights           0
neighbourhood_Castro/Upper Market      0
neighbourhood_Chinatown                0
neighbourhood_Crocker Amazon           0
neighbourhood_Diamond Heights          0
neighbourhood_Downtown/Civic Center    0
neighbourhood_Excelsio

In [15]:
#saving df_sflistings to a new CSV file
df_sflistings.to_csv('df_sflistings.csv', index=False)

#saving df_nyclistings to a new CSV file
df_nyclistings.to_csv('df_nyclistings.csv', index=False)